# 特徴量エンジニアリング（欠損値）

In [ ]:
!git clone https://github.com/ironerumi/fe_workshop.git -q -b work

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
pd.set_option('display.max_columns', 200)

from google.colab import files

In [ ]:
# collaboratoryにLendingClub50000.csvをアップロードする。
#uploaded = files.upload() # ローカルからアップロードできるが今回は使用しない。

In [ ]:
# ファイルを読み込む
df = pd.read_csv('fe_workshop/dataset/LendingClub50000.csv')

In [5]:
df.head().T

,0,1,2,3,4
申込ID,1529851,1824764,403548,646411,552526
メンバーID,1793711,2126933,442721,799671,711946
年,2012,2012,2009,2011,2010
月,9,11,5,1,7
ローン申請額,16000,3600,19200,21000,4000
借り入れ目的（大分類）,debt_consolidation,debt_consolidation,wedding,credit_card,other
借り入れ目的（小分類）,Debt Consolidation / Final Wedding Exp,Crush Credit Cards,Consolidate debt and pay for wedding,Wells BofA Credit Card Refi,Debt Consolidation
勤務先,PwC,Morgan Stanley Smith Barney,Aggregate Knowledge,Emerson Process Management,Home Depot
勤続年数,3,2,1,1,1
居住形態,RENT,RENT,RENT,RENT,RENT


In [6]:
# 一行ごとの欠損値の合計数を計算する。どのカラムが欠損で合計何カラムを確認。
nan_sum = df.isnull().sum(axis=1)
pd.merge(df.head().loc[:, df.head().isna().any()], nan_sum.head().to_frame(name='nan_sum'), left_index=True, right_index=True)

,直近の滞納からの月数,採集記録からの月数,前回の悪評価からの月数,徴収総額,全口座残高,nan_sum
0,NaN,NaN,NaN,0.0,164787.0,3
1,NaN,NaN,NaN,8806.0,96627.0,3
2,NaN,NaN,NaN,NaN,NaN,5
3,NaN,NaN,NaN,NaN,NaN,5
4,23.0,NaN,NaN,NaN,NaN,4


In [7]:
# 一行ごとの欠損値のパターンを文字列として取得する。
nan_pattern = df.apply(lambda x: ''.join(x.isnull().astype(int).astype(str)), axis=1)
nan_pattern.head()

0    0000000000000000000110000000100000
1    0000000000000000000110000000100000
2    0000000000000000000110000000100110
3    0000000000000000000110000000100110
4    0000000000000000000010000000100110
dtype: object

In [8]:
# 元データに追加する。
df['nan_sum'] = nan_sum
df['nan_pattern'] = nan_pattern

In [9]:
df.head().T

,0,1,2,3,4
申込ID,1529851,1824764,403548,646411,552526
メンバーID,1793711,2126933,442721,799671,711946
年,2012,2012,2009,2011,2010
月,9,11,5,1,7
ローン申請額,16000,3600,19200,21000,4000
借り入れ目的（大分類）,debt_consolidation,debt_consolidation,wedding,credit_card,other
借り入れ目的（小分類）,Debt Consolidation / Final Wedding Exp,Crush Credit Cards,Consolidate debt and pay for wedding,Wells BofA Credit Card Refi,Debt Consolidation
勤務先,PwC,Morgan Stanley Smith Barney,Aggregate Knowledge,Emerson Process Management,Home Depot
勤続年数,3,2,1,1,1
居住形態,RENT,RENT,RENT,RENT,RENT


In [ ]:
df.to_csv('LendingClub50000_nan.csv', index=False)

In [ ]:
# ダウンロードしたら、DataRobotでもう一度予測してみよう
files.download('LendingClub50000_nan.csv')